In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [59]:
df = pd.read_csv("/home/shigilsasi/code/Guvi_Projects/Axis_Bank_Intelligent_Product_&_Offer_Recommendation_System_using_Machine_Learning/output/axis_transactions.csv")
df.head()

,account_id,date,narration,channel,merchant,category,debit,credit,balance,source_file
0,0001_Thiyagarajan_Subbu_Statement,2022-12-01,UPI/973571737/LANDLORD RENT/landlord@okhdfc,UPI,LANDLORD RENT,Rent,15139.0,0.0,203044.0,0001_Thiyagarajan_Subbu_Statement.pdf
1,0001_Thiyagarajan_Subbu_Statement,2022-12-01,POS/988002/AMAZON INDIA/CARD **9369,POS,AMAZON INDIA,Shopping,3603.0,0.0,199441.0,0001_Thiyagarajan_Subbu_Statement.pdf
2,0001_Thiyagarajan_Subbu_Statement,2022-12-01,UPI/957858928/RAPIDO/rapido@ybl,UPI,RAPIDO,Transport,135.0,0.0,199306.0,0001_Thiyagarajan_Subbu_Statement.pdf
3,0001_Thiyagarajan_Subbu_Statement,2022-12-03,POS/214232/FLIPKART INDIA/CARD **9768,POS,FLIPKART INDIA,Shopping,1998.0,0.0,197308.0,0001_Thiyagarajan_Subbu_Statement.pdf
4,0001_Thiyagarajan_Subbu_Statement,2022-12-04,UPI/233699231/EATSURE/eatsure@hdfcbank,UPI,EATSURE,Other,374.0,0.0,196934.0,0001_Thiyagarajan_Subbu_Statement.pdf


In [60]:
df.columns

Index(['account_id', 'date', 'narration', 'channel', 'merchant', 'category',
       'debit', 'credit', 'balance', 'source_file'],
      dtype='object')

In [61]:
df.dtypes

account_id      object
date            object
narration       object
channel         object
merchant        object
category        object
debit          float64
credit         float64
balance        float64
source_file     object
dtype: object

In [62]:
df['merchant'].value_counts()

merchant
ZOMATO             85215
SWIGGY             84729
EATSURE            84617
OLA                56315
RAPIDO             56232
                   ...  
ACNO 9720363461        1
SIP909387              1
VL66096                1
ACNO 9087325699        1
SIP779180              1
Name: count, Length: 135880, dtype: int64

In [63]:
df['category'].unique()

array(['Rent', 'Shopping', 'Transport', 'Other', 'Food & Dining',
       'Entertainment', 'Investment', 'Utilities', 'Bank Charges',
       'ATM Cash Withdrawal', 'Salary', 'Interest Credit', 'Transfer',
       'Insurance', 'Fuel'], dtype=object)

In [64]:
df_other = df[df['category'] == 'Other']
df_other.head()

,account_id,date,narration,channel,merchant,category,debit,credit,balance,source_file
4,0001_Thiyagarajan_Subbu_Statement,2022-12-04,UPI/233699231/EATSURE/eatsure@hdfcbank,UPI,EATSURE,Other,374.0,0.0,196934.00,0001_Thiyagarajan_Subbu_Statement.pdf
5,0001_Thiyagarajan_Subbu_Statement,2022-12-04,UPI/503872733/SWIGGY/swiggy@hdfcbank,UPI,SWIGGY,Other,341.0,0.0,196593.00,0001_Thiyagarajan_Subbu_Statement.pdf
12,0001_Thiyagarajan_Subbu_Statement,2022-12-11,UPI/677898986/ZOMATO/zomato@hdfcbank,UPI,ZOMATO,Other,417.0,0.0,177592.45,0001_Thiyagarajan_Subbu_Statement.pdf
15,0001_Thiyagarajan_Subbu_Statement,2022-12-15,UPI/266011009/SPAR HYPERMARKET CBE/sparhyperma...,UPI,SPAR HYPERMARKET CBE,Other,3334.0,0.0,167370.45,0001_Thiyagarajan_Subbu_Statement.pdf
16,0001_Thiyagarajan_Subbu_Statement,2022-12-15,UPI/635472868/BOOKMYSHOW/bms@axisbank,UPI,BOOKMYSHOW,Other,573.0,0.0,166797.45,0001_Thiyagarajan_Subbu_Statement.pdf


In [65]:
df_other.sample(100)

,account_id,date,narration,channel,merchant,category,debit,credit,balance,source_file
246094,0178_Murali_Arjun_Statement,2022-04-10,UPI/424861413/SWIGGY/swiggy@hdfcbank,UPI,SWIGGY,Other,743.0,0.0,587689.90,0178_Murali_Arjun_Statement.pdf
109211,0082_Chitralekha_Karthik_Statement,2022-08-12,UPI/828968434/NILGIRIS CBE/nilgiriscbe@upi,UPI,NILGIRIS CBE,Other,1250.0,0.0,996133.45,0082_Chitralekha_Karthik_Statement.pdf
795936,0570_Hari_Srinivas_Statement,2025-08-05,ACH/DEBIT/ACT FIBERNET/ACTBB45217036,ACH,ACT FIBERNET,Other,599.0,0.0,1420070.82,0570_Hari_Srinivas_Statement.pdf
1246160,0897_Mani_Prabhu_Statement,2025-06-05,ACH/DEBIT/ACT FIBERNET/ACTBB62685223,ACH,ACT FIBERNET,Other,599.0,0.0,-1551948.12,0897_Mani_Prabhu_Statement.pdf
761116,0544_Thenmozhi_Durai_Statement,2025-04-19,UPI/771043808/KOVAI PAZHAMUDIR/kovaipazhamudir...,UPI,KOVAI PAZHAMUDIR,Other,2359.0,0.0,3082205.06,0544_Thenmozhi_Durai_Statement.pdf
...,...,...,...,...,...,...,...,...,...,...
52814,0038_Padma_Siva_Statement,2025-01-24,POS/688813/SHREE ANANDHAAS/CARD **5171,POS,SHREE ANANDHAAS,Other,1567.0,0.0,233502.93,0038_Padma_Siva_Statement.pdf
1168470,0843_Saraswathi_Raju_Statement,2025-03-19,UPI/944991938/APOLLO PHARMACY CBE/apollopharma...,UPI,APOLLO PHARMACY CBE,Other,366.0,0.0,112673.79,0843_Saraswathi_Raju_Statement.pdf
380246,0268_Balaji_Samy_Statement,2021-08-01,UPI/508675767/SWIGGY/swiggy@hdfcbank,UPI,SWIGGY,Other,680.0,0.0,204836.00,0268_Balaji_Samy_Statement.pdf
249989,0180_Lakshman_Raj_Statement,2021-09-28,UPI/695165712/EATSURE/eatsure@hdfcbank,UPI,EATSURE,Other,283.0,0.0,707264.12,0180_Lakshman_Raj_Statement.pdf


In [66]:
df_other['merchant'].value_counts()

merchant
ZOMATO                85215
SWIGGY                84729
EATSURE               84617
ACT FIBERNET          23412
RELIANCE FRESH CBE    17077
                      ...  
LN68991                   1
LN89667                   1
LN29412                   1
LN76328                   1
LN11866                   1
Name: count, Length: 19215, dtype: int64

In [67]:
FOOD_MERCHANTS = [
    "zomato", "swiggy", "eatsure", "restaurant", "hotel",
    "biryani", "mess", "annandhaas", "annapoorna"
]

GROCERY_MERCHANTS = [
    "reliance fresh", "spar", "pazhamudir", "nilgiris",
    "supermarket", "hypermarket"
]

TRANSPORT_MERCHANTS = ["ola", "uber", "rapido"]


In [68]:
def recategorize_other(row):
    text = f"{row['merchant']} {row['narration']}".lower()

    # Food & Dining
    if any(m in text for m in FOOD_MERCHANTS):
        return "Food & Dining"

    # Groceries
    if any(m in text for m in GROCERY_MERCHANTS):
        return "Groceries"

    # Transport
    if any(m in text for m in TRANSPORT_MERCHANTS):
        return "Transport"

    # Utilities
    if any(x in text for x in ["electricity", "water", "gas", "tangedco", "billpay"]):
        return "Utilities"

    # Rent
    if "rent" in text or "landlord" in text:
        return "Rent"

    # Investment
    if "sip" in text or "mutual fund" in text:
        return "Investment"

    # Bank Charges
    if "charge" in text or "sms alert" in text:
        return "Bank Charges"

    # Interest
    if "interest" in text or "int/credit" in text:
        return "Interest Credit"

    # Transfers
    if any(x in text for x in ["upi", "neft", "imps", "rtgs"]):
        return "Transfer"

    return "Unknown Merchant"


In [69]:
mask = df['category'] == 'Other'
df.loc[mask, 'category'] = df[mask].apply(recategorize_other, axis=1)
df['category'].value_counts()

category
Food & Dining          290831
Shopping               189742
Transport              168756
ATM Cash Withdrawal    135264
Unknown Merchant       106302
Groceries              101670
Transfer                99452
Utilities               70852
Fuel                    58628
Bank Charges            45408
Investment              34584
Salary                  33807
Rent                    17383
Entertainment           16852
Interest Credit         11245
Insurance                3259
Name: count, dtype: int64

In [70]:
import re

def normalize_merchant(row):
    text = row['narration'].lower()
    merchant = row['merchant']

    # Electricity
    if "tangedco" in text:
        return "TANGEDCO"

    # SIP / Mutual Fund
    if text.startswith("sip") or "mutual fund" in text:
        return "AXIS MUTUAL FUND"

    # ATM
    if row['category'] == "ATM Cash Withdrawal":
        return "ATM Withdrawal"

    # Bank Charges
    if row['category'] == "Bank Charges":
        return "Axis Bank"

    # Interest
    if row['category'] == "Interest Credit":
        return "Axis Bank"

    # Transfers
    if row['category'] == "Transfer":
        return "Personal Transfer"

    # Remove ACNO / numeric junk
    if re.search(r'acno|\d{6,}', merchant.lower()):
        return "Utility / Reference"

    return merchant.title()


In [71]:
df['clean_merchant'] = df.apply(normalize_merchant, axis=1)

In [72]:
df['merchant'] = df['clean_merchant']
df.drop(columns=['clean_merchant'], inplace=True)

In [73]:
def normalize_system_transactions(row):
    text = row['narration'].lower()

    # Credit Card Bill Payment
    if "axis credit card" in text:
        row['category'] = "Credit Card Payment"
        row['merchant'] = "Axis Credit Card"
        return row

    # Home Loan EMI
    if text.startswith("emi") and "home loan" in text:
        row['category'] = "Loan EMI"
        row['merchant'] = "Axis Home Loan"
        return row

    # Car Loan EMI
    if text.startswith("emi") and "car loan" in text:
        row['category'] = "Loan EMI"
        row['merchant'] = "Axis Car Loan"
        return row

    return row


In [74]:
df = df.apply(normalize_system_transactions, axis=1)

In [75]:
def normalize_emi_transactions(row):
    text = str(row['narration']).lower()
    merchant = str(row['merchant']).upper()

    if text.startswith("emi"):
        row['category'] = "Loan EMI"

        if merchant.startswith("PL"):
            row['merchant'] = "Axis Personal Loan"

        elif merchant.startswith("VL"):
            row['merchant'] = "Axis Vehicle Loan"

        elif merchant.startswith("LN"):
            row['merchant'] = "Axis Home Loan"

        else:
            row['merchant'] = "Axis Loan"

    return row

In [76]:
df = df.apply(normalize_emi_transactions, axis=1)

In [82]:
import re

def normalize_salary_transactions(row):
    text = str(row['narration']).upper()

    if row['category'] == "Salary" and "SALARY" in text:
        # Extract employer name between NEFT-xxxx- and -SALARY
        match = re.search(r'NEFT-\d+-(.*?)-SALARY', text)
        if match:
            employer = match.group(1).title()
            row['merchant'] = employer
        else:
            row['merchant'] = "Employer"

    return row


In [83]:
df = df.apply(normalize_salary_transactions, axis=1)

In [84]:
df['merchant'].value_counts()

merchant
ATM Withdrawal         135264
Personal Transfer       99452
Zomato                  85215
Swiggy                  84729
Eatsure                 84617
                        ...  
Tcs India Pvt Ltd         651
Infosys Limited           608
Suguna Foods              597
Coimbatore Textiles       559
Mindtree Ltd              513
Name: count, Length: 86, dtype: int64

In [85]:
df['category'].value_counts()   

category
Food & Dining          290831
Shopping               189742
Transport              168756
ATM Cash Withdrawal    135264
Groceries              101670
Transfer                99452
Unknown Merchant        86181
Fuel                    58628
Utilities               50637
Bank Charges            45408
Investment              34584
Salary                  33807
Credit Card Payment     20215
Loan EMI                20121
Rent                    17383
Entertainment           16852
Interest Credit         11245
Insurance                3259
Name: count, dtype: int64

In [86]:
df.head()

,account_id,date,narration,channel,merchant,category,debit,credit,balance,source_file
0,0001_Thiyagarajan_Subbu_Statement,2022-12-01,UPI/973571737/LANDLORD RENT/landlord@okhdfc,UPI,Landlord Rent,Rent,15139.0,0.0,203044.0,0001_Thiyagarajan_Subbu_Statement.pdf
1,0001_Thiyagarajan_Subbu_Statement,2022-12-01,POS/988002/AMAZON INDIA/CARD **9369,POS,Amazon India,Shopping,3603.0,0.0,199441.0,0001_Thiyagarajan_Subbu_Statement.pdf
2,0001_Thiyagarajan_Subbu_Statement,2022-12-01,UPI/957858928/RAPIDO/rapido@ybl,UPI,Rapido,Transport,135.0,0.0,199306.0,0001_Thiyagarajan_Subbu_Statement.pdf
3,0001_Thiyagarajan_Subbu_Statement,2022-12-03,POS/214232/FLIPKART INDIA/CARD **9768,POS,Flipkart India,Shopping,1998.0,0.0,197308.0,0001_Thiyagarajan_Subbu_Statement.pdf
4,0001_Thiyagarajan_Subbu_Statement,2022-12-04,UPI/233699231/EATSURE/eatsure@hdfcbank,UPI,Eatsure,Food & Dining,374.0,0.0,196934.0,0001_Thiyagarajan_Subbu_Statement.pdf
